In [266]:
## from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import os
import io

import os

import sys

import string

import numpy as np

import pandas as pd

from tensorflow import keras

from __future__ import print_function

from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding


#
# with open(f"{cwd}\inspiration\{dataroot[0]}").read().replace('\n', '. ') as file:
#     print(file)
cwd = os.getcwd()
dataroot = os.listdir(cwd + "\inspiration")


translator = str.maketrans('', '', string.punctuation)
sentData = []


for i in range(0, len(dataroot)):
    print(f"{cwd}\inspiration\{dataroot[i]}")
    data2 = open(f"{cwd}\inspiration\{dataroot[i]}", encoding="utf8").read().replace(',', '').replace('\n', ' \n ').replace("'","").replace('(','').replace(')','').lower()
    sentData.append(data2)



C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Andre 3000_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\ASAP Rocky_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\business.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Drake_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\eminemlyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\getIntoIt.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Isaiah Rashad_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\J Cole_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Logic_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Mac Miller_lyrics.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\melt_session.txt
C:\Users\sebbe\Desktop\MED-local\ElectricWordsmith\inspiration\Nas_lyrics.txt
C:\Users\sebbe\Desktop\ME

In [267]:
print(len(sentData))
print(sentData[11][:])


14
 
 yeah yeah 
 ayo black its time word word its time man 
 its time man aight man begin 
 yeah straight out the fuckin dungeons of rap 
 where fake niggas dont make it back 
 i dont know how to start this shit yo... now 
  
 rappers i monkey flip em with the funky rhythm i be kickin 
 musician inflictin composition of pain 
 im like scarface sniffin cocaine 
 holdin an m16 see with the pen im extreme – now 
 bullet holes left in my peepholes im suited up in street clothes 
 hand me a 9 and ill defeat foes 
 yall know my steelo with or without the airplay 
 i keep some e&j sittin bent up in the stairway 
 or either on the corner bettin grants with the cee-lo champs 
 laughin at baseheads tryna sell some broken amps 
 g-packs get off quick forever niggas talk shit 
 reminiscin about the last time the task force flipped 
 niggas be runnin through the block shootin 
 time to start the revolution catch a body head for houston 
 once they caught us off-guard the mac-10 was in the grass an

In [269]:
textAsList = []
textAsList.clear()

def extractTextAsList(text):
    global textAsList
    textAsList += [w for w in text.split(' ') if w.strip() !='' or w=='\n' or w=='.']
    
    
        
    


In [270]:
for index in range(0, len(sentData)):
        extractTextAsList(sentData[index])
        
print("Total amount of words: ", len(textAsList))
print(textAsList[100])

Total amount of words:  441048




In [275]:
frequencies = {}
frequencies.clear()


for word in textAsList:
    frequencies[word] = frequencies.get(word, 0) + 1
    
print(len(frequencies))

print("How many fucks?:", frequencies['fuck']+ frequencies['fucking'] + frequencies['fucker'] + frequencies['fucks'] + frequencies['fuckers'])
print("How many nibbas?: ", frequencies['nigga'] + frequencies['niggas']+ frequencies['nigger'] + frequencies['niggers'])

22895
How many fucks?: 2171
How many nibbas?:  2778


In [276]:
uncommonWords = set()
minFreq = 7
minSeq = 5
BATCH_SIZE = 32

uncommonWords = set([key for key in frequencies.keys() if frequencies[key] < minFreq])
words = sorted(set([key for key in frequencies.keys() if frequencies[key] >= minFreq]))

numWords = len(words)

wordIndices = dict((w, i) for i, w in enumerate(words))
indicesWord = dict((i, w) for i, w in enumerate(words))

#print(wordIndices)
#print(indicesWord)

print(f'Words with less than {minFreq} appearances: {len(uncommonWords)}')

print(f'Words with more than {minFreq} appearances: {len(words)}')

validSeqs = []
endSeqWords = []

for i in range(len(textAsList) - minSeq):
    endSlice = i + minSeq + 1
    
    if len(set(textAsList[i: endSlice]).intersection(uncommonWords)) == 0:
        validSeqs.append(textAsList[i: i + minSeq])
        endSeqWords.append(textAsList[i + minSeq])
        
print(f'Valid sequences of size {minSeq}: {len(validSeqs)}')

Words with less than 7 appearances: 19067
Words with more than 7 appearances: 3828
Valid sequences of size 5: 285412


In [282]:
xTrain, xTest, yTrain, yTest = train_test_split(validSeqs, endSeqWords, test_size=0.02, random_state=42)

print(xTrain[2:5])

# Data generator for fit and evaluate
def generator(sentenceList, nextWordList, batchSize):
    index = 0
    
    while True:
        x = np.zeros((batchSize, minSeq), dtype=np.int32)
        y = np.zeros((batchSize), dtype=np.int32)
        
        for i in range(batchSize):
            for t, w in enumerate(sentenceList[index % len(sentenceList)]):
                x[i, t] = wordIndices[w]
                
            y[i] = wordIndices[nextWordList[index % len(sentenceList)]]
            
            
            index = index + 1
            
        yield x, y

# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    #helper function to sample and index from a probabillity array
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    
    expPreds = np.exp(preds)
    
    preds = expPreds / np.sum(expPreds)
    
    probabIndx = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probabIndx)

def onEpochEnd(epoch, logs):
    
    #Function invoked at end of each epoch. Prints generated text.
    fxFile.write('\n----- Generating text after Epoch: %d\n' % epoch)
    
    #Pick seed at random index
    seedIndex = np.random.randint(len(xTrain+xTest))
    seed = (xTrain+xTest)[seedIndex]
    
    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        
        fxFile.write('----- Diversity: ' + str(diversity) + '\n')
        fxFile.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        fxFile.write(' '.join(sentence))
        
        for i in range(50):
            xPred = np.zeros((1, minSeq))
            
            for t, word in enumerate(sentence):
                
                xPred[0, t] = wordIndices[word]
                
            preds = model.predict(xPred, verbose=0)[0]
            
            nextIndex = sample(preds, diversity)
            
            nextWord = indicesWord[nextIndex]
            
            sentence = sentence[1:]
            
            sentence.append(nextWord)
            
            fxFile.write('\n')
        
        fxFile.write('='*80 + '\n')
        fxFile.flush()
    
            

[['life', 'out', 'of', 'the', 'question'], ['a', 'leg', 'quite', 'like', 'this'], ['i', 'dont', 'know', 'anyone', '\n']]


In [283]:
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding

def get_model():
    print('Build model...')
    
    model = Sequential()
    
    model.add(Embedding(input_dim=len(words), output_dim=1024))
    
    model.add(Bidirectional(LSTM(128)))
    
    model.add(Dense(len(words)))
    
    model.add(Activation('softmax'))
    
    return model
    
    

In [284]:
model = get_model()

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
"loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}" % \
(len(words), minSeq, minFreq)

Build model...


In [285]:
checkpoint = keras.callbacks.ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)

print_callback = LambdaCallback(on_epoch_end=onEpochEnd)

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20)

callbackList = [checkpoint, print_callback, earlyStopping]

fxFile = open('examples.txt', 'w')

model.fit(generator(xTrain, yTest, BATCH_SIZE),
         steps_per_epoch=int(len(validSeqs)/BATCH_SIZE)+1,
         epochs=20)

Epoch 1/20
 178/8920 [..............................] - ETA: 16:02 - loss: 6.4505 - accuracy: 0.1004

UnknownError: Graph execution error:

IndexError: list index out of range
Traceback (most recent call last):

  File "C:\Users\sebbe\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\Users\sebbe\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\sebbe\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\sebbe\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 830, in wrapped_generator
    for data in generator_fn():

  File "<ipython-input-282-9010b7b36ac0>", line 17, in generator
    y[i] = wordIndices[nextWordList[index % len(sentenceList)]]

IndexError: list index out of range


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_159266]